# Website Instructions

## To rebuild site and deploy

Run this and it will convert blog posts to html and push to google cloud.  It may take a few hours for the changes to reflect on the website.

## To add a blog post

+ The notebook must be named in this format `yyyy-mm-dd-title`
+ The first cell must be markdown and formatted in a particular way with particular information.  Look at the first cell in an existing blog post as reference.
+ Blog posts are jupyter notebooks stored in `website/posts/notebooks/`.  Move any notebooks there - this notebook will pick them up and do the conversion to html for you.  For example: `cp WIP/example.ipynb website/posts/notebooks/2021-12-31-blogtitle.ipynb`

In [11]:
import os
from fastcore.foundation import *
from pathlib import Path

In [12]:
posts_path = Path('./website/posts/')
html_path = posts_path/'html'
posts_path,html_path

(PosixPath('website/posts'), PosixPath('website/posts/html'))

In [13]:
def nb_to_post(input_path=posts_path/'notebooks',output_path=html_path):
    os.system(f"rm ./{output_path}/*.html")
    os.system(f'jupyter nbconvert --to html {input_path}/*.ipynb') # eventually --template basic then css style
    os.system(f"mv ./{input_path}/*.html ./{output_path}/")
    
nb_to_post()

[NbConvertApp] Converting notebook website/posts/notebooks/2021-09-25-527 IRS Filings Data.ipynb to html
[NbConvertApp] Writing 633492 bytes to website/posts/notebooks/2021-09-25-527 IRS Filings Data.html


In [14]:
posts = L(os.listdir(html_path))
posts.sort(reverse=True)     

In [15]:
for post in [o for o in posts if o[-5:]=='.html']:

    with open(html_path/post) as file: htmlfile = file.read()

    section1 = htmlfile[:htmlfile.find("</body>")]
    section2 = htmlfile[htmlfile.find("</body>"):]
    new_section = '''\n\n<script src="postformatting.js"></script>\n\n'''

    new_html = section1+new_section+section2
    
    with open(html_path/post,'w') as file: file.write(new_html)

In [16]:
def get_slice(file,start_str,end_str):
    with open(file) as f:
        tmp = f.read()    
    slice1 = tmp[tmp.find(start_str)+len(start_str):]
    out = slice1[:slice1.find(end_str)]
    return out

In [17]:
htmls = []
for post in posts:
    if post[-5:] != '.html': continue
    date = post[:10]
    
    name_tmp = post[11:-5]
    name = ''
    for i, letter in enumerate(name_tmp):
        if i and letter.isupper() and not name_tmp[i-1].isupper():
            name += ' '
        
        name += letter  
    description = get_slice(html_path/post,'Post Description:</strong> ','</p>')
    tags = get_slice(html_path/post,'Post Categories:</strong> ','</p>')

    htmls.append('<div class="smallblogdiv">')
    htmls.append(f'''<a class="postlink" href="./posts/html/{post}">{name}</a>''')
    htmls.append(f'''<p class="postdescription">{description}</p>''')
    htmls.append(f'''<p class="posttags">Tags: {tags}</p>''')
    htmls.append(f'''<p class="postdate">{date}</p>''')
    htmls.append('''</div>''')
htmls = '\n              '.join(htmls)

In [18]:
with open(posts_path/"blog_template.html") as f: tmp = f.read()
tmp = tmp.replace('AUTOINSERTIONPOINT',htmls)
with open(Path("website/blog.html"),'w') as f: f.write(tmp)

In [19]:
!gsutil -m rsync -d -r website/ gs://vinculum.solutions/



Updates are available for some Cloud SDK components.  To install them,
please run:
  $ gcloud components update

Building synchronization state...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Starting synchronization...
If you experience problems with multiprocessing on MacOS, they might be related to https://bugs.python.org/issue33725. You can disable multiprocessing by editing your .boto config or by adding the following flag to your command: `-o "GSUtil:parallel_process_count=1"`. Note that multithreading is still available even if you disable multiprocessing.

Removing gs://vinculum.solutions/archive_projects.html
Copying file://website/assets.html [Content-Type=text/html]...
Cop